In [2]:
import pandas as pd
import numpy as np
import itertools
import json
import copy

class LitBot:
    EPSILON = 0.01
    def __init__(self, team_id, player_id, player_count, game_state_array=None, fake_game_seed = None):

        self.team_id = team_id
        self.player_id = player_id
        self.player_count = player_count

        if fake_game_seed is not None:
            (
                self._fake_initial_game_state_array,
                self._fake_initial_card_location_array
            ) =  self.initialize_fake_game(fake_game_seed)
            self.initialize_matrix(self.team_id, self.player_id, self._fake_initial_game_state_array[0, 0])
        elif game_state_array is not None:
            self.initialize_matrix(self.team_id, self.player_id, game_state_array)
    
    def initialize_fake_game(self, fake_game_seed):
        
        np.random.seed(fake_game_seed)
        card_location_array = np.random.choice(range(self.player_count), (8,6))
        card_location_array_team = card_location_array%2
        card_location_array_player = card_location_array//2

        game_state_array = np.zeros((2, self.player_count//2, 8, 6))
        for set_id, card_id in itertools.product(range(8), range(6)):
            game_state_array[
                card_location_array_team[set_id, card_id],
                card_location_array_player[set_id, card_id],
                set_id,        
                card_id
            ] = 1
        
        return game_state_array, card_location_array
    
    def dynamic_state(self, team_id, player_id, set_id, card_id, inference):

        return 1

    def initialize_matrix(self, team_id, player_id, player_array):

        self.filler = np.full((8,6), self.dynamic_state)

        self.information_matrix = np.full((2, self.player_count//2, 8, 6), LitBot.EPSILON)
        for team_idx, player_idx in itertools.product(range(2), range(self.player_count//2)):
            if (team_idx == team_id) and (player_idx == player_id):
                self.information_matrix[team_idx, player_idx, :, :] = np.where(player_array == 1, 1, 0)
            else:
                self.information_matrix[team_idx, player_idx, :, :] = np.where(player_array == 1, 0, LitBot.EPSILON)

        self.player_set_card_count = np.full((2, self.player_count//2, 8), 0)
        self.player_set_card_count[team_id, player_id, :] =  player_array.sum(axis=1)
        
        self.player_card_count = np.full((2, self.player_count//2), 48//self.player_count)
                
        self.set_card_count = np.full((8), 6)

        self.active_sets = np.full((8), LitBot.EPSILON) # active 1 closed sets keep scores here
        self.active_cards = np.full((8,6), LitBot.EPSILON)
        self.recent_card_array = np.full((2,8), LitBot.EPSILON)
        
        self.inferences_list = {
            0 : [],
            1 : [],
            2 : [],
            3 : [],
            4 : [],
            5 : [],
            6 : [],
            7 : []
        }  # inference will be stored as {active cards array copy : 1}

    def inv_team_id(self, team_id):
        return int(not bool(team_id))


    def update_information_matrix_hard(self, card_info, ask_player, ans_player, result):

        if result == 0:
            assert ans_player is not None
            self.information_matrix[ask_player["team_id"], ask_player["player_id"], card_info["set_id"], card_info["card_id"]] = 0
            self.information_matrix[ans_player["team_id"], ans_player["player_id"], card_info["set_id"], card_info["card_id"]] = 0

        if result == 1:
            self.information_matrix[ask_player["team_id"], ask_player["player_id"], card_info["set_id"], card_info["card_id"]] = 1
            self.information_matrix[ask_player["team_id"], [i for i in np.arange(self.player_count//2) if i != ask_player["player_id"]], card_info["set_id"], card_info["card_id"]] = 0
            self.information_matrix[self.inv_team_id(ask_player["team_id"]), :, card_info["set_id"], card_info["card_id"]] = 0

    @property
    def scores(self):
        return np.array([3,2])

    def update_active_arrays(self, team_id, set_id, card_id, action, result):
        if action == "ask_card":
            assert card_id is not None
            if result == 0:
                self.active_cards[set_id, card_id] = 1
                self.recent_card_array[team_id, set_id] = card_id
            else:
                self.active_cards[set_id, card_id] = 0
            self.active_sets[set_id] = 1
        if action == "card":
            self.active_cards[set_id, :] = 0
            self.active_sets[set_id] = 0

    def update_inference_list(self, team_id, player_id, set_id, card_id):

        if self.recent_card_array[team_id, set_id] != card_id:
            self.inferences_list.append(
                {
                    "team_id" : team_id, 
                    "player_id" : player_id, 
                    "set_id" : set_id, 
                    "card_id" : card_id
                }
            )


    def update_game(self, game_action_dict, stop = True):

        action = game_action_dict["action"]
        team_id = game_action_dict["by_team"]
        player_id = game_action_dict["by"]
        set_id = game_action_dict["set_id"]
        result = game_action_dict["result"]

        ans_team_id = game_action_dict["to_team"]
        ans_player_id = game_action_dict["to"]
        card_id = game_action_dict["card_id"]
        card_locations = game_action_dict["card_locations"]

        if action == "ask_card":

            self.update_information_matrix_hard(
                card_info = {"set_id" : set_id, "card_id" : card_id},
                ask_player = {"team_id" : team_id, "player_id" : player_id},
                ans_player = {"team_id" : ans_team_id, "player_id" : ans_player_id},
                result = result
            )
        elif action == "call_set":

            for card_id, player_id in card_locations.items():
                card_id = int(card_id)

                self.update_information_matrix_hard(
                    card_info = {"set_id" : set_id, "card_id" : card_id},
                    ask_player = {"team_id" : team_id, "player_id" : player_id},
                    ans_player = None,
                    result = 1
                )
        else:
            pass
        
        self.update_inference_list(team_id, player_id, set_id, card_id)
        self.update_active_arrays(team_id, set_id, card_id, action, result)

        
        if game_action_dict["action"] == "ask_card":

            ask_team_id = game_action_dict["by_team"]
            ans_team_id = game_action_dict["to_team"]
            ask_player_id = game_action_dict["by"]
            ans_player_id = game_action_dict["to"]
            set_id = game_action_dict["set_id"]
            card_id = game_action_dict["card_id"]
            result = game_action_dict["result"]

            if result == 1:
                print("updated truth matrix 1")
                self.truth_matrix[ask_team_id, ask_player_id, set_id, card_id] = 1
                self.truth_matrix[ask_team_id, [i for i in np.arange(self.player_count//2) if i != ask_player_id ], set_id, card_id] = 0
                self.truth_matrix[ans_team_id, [i for i in np.arange(self.player_count//2)], set_id, card_id] = 0

                print("updated inference matrix 1")
                self.inference_matrix[ask_team_id, ask_player_id, set_id, card_id] = 1
                self.inference_matrix[ask_team_id, [i for i in np.arange(self.player_count//2) if i != ask_player_id ], set_id, card_id] = 0
                self.inference_matrix[ans_team_id, [i for i in np.arange(self.player_count//2)], set_id, card_id] = 0


            else:
                print("updated truth matrix 0")
                self.truth_matrix[ask_team_id, ask_player_id, set_id, card_id] = 0
                self.truth_matrix[ans_team_id, ans_player_id, set_id, card_id] = 0
                print("updated inference matrix 0")
                self.inference_matrix[ask_team_id, ask_player_id, set_id, card_id] = 0
                self.inference_matrix[ans_team_id, ans_player_id, set_id, card_id] = 0
                
                self.update_inference_matrix(ask_player_id, ask_team_id, set_id, card_id)

            self.update_player_card_count(ask_team_id, ask_player_id, ans_team_id, ans_player_id, set_id, card_id, result)


            self.update_active_sets(set_id, card_id, result)
            self.recent_card_array[set_id, 0] = card_id
            self.recent_card_array[set_id, 1] = ask_team_id
            self.prob_matrix = self.update_prob_matrix(self.truth_matrix)
            # self.shannon_info_matrix = self.update_shannon_info_matrix(self.prob_matrix)
        
        if game_action_dict["action"] == "call_set":


            {'action': 'call_set',
            'by_team': 0,
            'by': 2,
            'set_id': 4,
            'card_locations': {'0': 1, '1': 0, '2': 2, '3': 0, '4': 1, '5': 2},
            'result': 1},

            team_id = game_action_dict["by_team"]
            set_id = game_action_dict["set_id"]
            card_locations = game_action_dict["card_locations"]

            for card_id, player_id in card_locations.items():
                print("updated truth matrix callset")
                self.truth_matrix[team_id, player_id, set_id, int(card_id)] = 1
                self.truth_matrix[team_id, [i for i in np.arange(self.player_count//2) if i != player_id ], set_id, int(card_id)] = 0
                self.truth_matrix[0 if team_id==0 else 1, [i for i in np.arange(self.player_count//2)], set_id, int(card_id)] = 0

                print("updated inference matrix callset")
                self.inference_matrix[team_id, player_id, set_id, int(card_id)] = 1
                self.inference_matrix[team_id, [i for i in np.arange(self.player_count//2) if i != player_id ], set_id, int(card_id)] = 0
                self.inference_matrix[0 if team_id==0 else 1, [i for i in np.arange(self.player_count//2)], set_id, int(card_id)] = 0


In [3]:
bot1 = LitBot(0,0, 6,fake_game_seed=0)